In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

### Caricamento dataset, stampe e prima pulizia dei dati

In [2]:
telco_churn_file_path = 'WA_Fn-UseC_-Telco-Customer-Churn.train.csv'
telco_churn_data = pd.read_csv(telco_churn_file_path)

In [3]:
telco_churn_data.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,5634.000000,5634.000000,5634.000000
mean,0.160809,32.373092,64.864253
std,0.367388,24.424539,30.089324
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.750000
50%,0.000000,29.000000,70.525000
75%,0.000000,55.000000,89.937500
max,1.000000,72.000000,118.600000


In [4]:
telco_churn_data.columns

Index(['Churn', 'customerID', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges'],
      dtype='object')

In [5]:
telco_churn_data.head()

,Churn,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,No,4223-BKEOR,Female,0,No,Yes,21,Yes,No,DSL,...,No,Yes,No,No,Yes,One year,No,Mailed check,64.85,1336.8
1,No,6035-RIIOM,Female,0,No,No,54,Yes,Yes,Fiber optic,...,Yes,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),97.20,5129.45
2,Yes,3797-VTIDR,Male,0,Yes,No,1,No,No phone service,DSL,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,23.45,23.45
3,Yes,2568-BRGYX,Male,0,No,No,4,Yes,No,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,237.95
4,No,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,


In [6]:
# ho notato su alcune colonne relative ai servizi, oltre ai valori yes/no, anche un terzo valore (no internet service/no phone service)
telco_churn_data[['MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies' ]]

,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,No,Yes,No,Yes,No,No,Yes
1,Yes,No,Yes,No,No,Yes,Yes
2,No phone service,No,No,No,No,No,No
3,No,No,No,No,No,No,No
4,Yes,Yes,Yes,No,Yes,No,No
...,...,...,...,...,...,...,...
5629,No,Yes,No,No,No,Yes,Yes
5630,Yes,Yes,Yes,Yes,Yes,Yes,Yes
5631,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service
5632,Yes,No,No,Yes,No,Yes,Yes


In [7]:
# Considero 'No phone service' e 'No internet service' equivalenti a 'No' (non hanno quel servizio)
internet_cols_replace = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in internet_cols_replace : 
    telco_churn_data[i]  = telco_churn_data[i].replace({'No internet service' : 'No'})
telco_churn_data['MultipleLines']  = telco_churn_data['MultipleLines'].replace({'No phone service' : 'No'})

In [8]:
# controllo che non ci siano valori nulli
telco_churn_data.isnull().sum()

Churn               0
customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
dtype: int64

In [9]:
# Ho notato nella colonna 'TotalCharges' degli spazi 
telco_churn_data['TotalCharges']

0        1336.8
1       5129.45
2         23.45
3        237.95
4              
         ...   
5629         95
5630     2198.3
5631     306.05
5632    1200.15
5633      457.3
Name: TotalCharges, Length: 5634, dtype: object

In [10]:
# conto quanti sono gli spazi
counts = telco_churn_data['TotalCharges'].value_counts().to_dict()
print (counts[" "])

10


In [11]:
# sostituisco NaN agli spazi, e converto il tipo della colonna in float (era object)
telco_churn_data['TotalCharges']=telco_churn_data["TotalCharges"].replace(" ",np.nan)
telco_churn_data["TotalCharges"] = telco_churn_data["TotalCharges"].astype(float)

In [12]:
# sostituisco ai valori nulli la media degli altri valori della colonna
telco_churn_data['TotalCharges'] = telco_churn_data['TotalCharges'].fillna(telco_churn_data['TotalCharges'].mean())

In [13]:
# controllo nuovamente che non ci siano valori nulli
telco_churn_data.isnull().sum()

Churn               0
customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
dtype: int64

In [14]:
# elimino la colonna 'customerID'
# non può essere considerata una feature, in quanto identificativo unico per ogni utente
telco_churn_data=telco_churn_data.drop(['customerID'], axis=1)

### Label Encoding

In [15]:
# individuo le colonne di tipo 'object'
s = (telco_churn_data.dtypes == 'object')
object_cols = list(s[s].index)
print (object_cols)

['Churn', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']


In [16]:
# copio il dataset ed effettuo il label encoding per ogni colonna di tipo 'object'
label_telco_churn_data = telco_churn_data.copy()
label_encoder = LabelEncoder()
for col in object_cols:
    label_telco_churn_data[col] = label_encoder.fit_transform(telco_churn_data[col])

In [17]:
label_telco_churn_data.head()

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0,0,0,0,1,21,1,0,0,1,0,1,0,0,1,1,0,3,64.85,1336.800000
1,0,0,0,0,0,54,1,1,1,0,1,0,0,1,1,2,1,0,97.20,5129.450000
2,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,2,23.45,23.450000
3,1,1,0,0,0,4,1,0,1,0,0,0,0,0,0,0,1,2,70.20,237.950000
4,0,1,0,0,1,0,1,1,0,1,1,0,1,0,0,2,1,0,61.90,2291.154605


In [18]:
label_telco_churn_data.tail()

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
5629,1,1,0,1,0,1,1,0,1,1,0,0,0,1,1,0,1,2,95.00,95.00
5630,0,0,0,1,1,23,1,1,0,1,1,1,1,1,1,2,1,1,91.10,2198.30
5631,0,1,0,1,1,12,1,0,2,0,0,0,0,0,0,0,1,2,21.15,306.05
5632,1,1,1,0,0,12,1,1,1,0,0,1,0,1,1,0,1,2,99.45,1200.15
5633,0,1,0,0,0,26,1,0,2,0,0,0,0,0,0,1,0,1,19.80,457.30


### Individuazione target e split dei dati

In [19]:
# seleziono la colonna target
y=label_telco_churn_data.Churn

In [20]:
y

0       0
1       0
2       1
3       1
4       0
       ..
5629    1
5630    0
5631    0
5632    1
5633    0
Name: Churn, Length: 5634, dtype: int32

In [21]:
# elimino la colonna target dal dataset
X=label_telco_churn_data.drop(['Churn'], axis=1)

In [22]:
# split dei dati
train_X,val_X,train_y,val_y=train_test_split(X,y,test_size=0.3,random_state=1)

In [23]:
train_X

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
821,0,1,1,0,66,1,1,1,0,1,1,0,1,1,1,0,1,104.60,6819.45
1571,0,0,1,1,64,1,1,1,1,0,0,1,0,0,0,1,0,86.80,5327.25
4355,0,0,0,0,14,1,0,1,0,1,0,1,0,0,0,1,1,80.05,1112.30
4709,1,0,1,0,40,1,1,0,0,1,0,1,1,0,1,0,1,70.75,2921.75
4799,0,0,1,1,25,1,0,2,0,0,0,0,0,0,2,0,3,20.10,486.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,1,1,1,0,55,1,1,1,0,1,1,1,1,1,2,1,2,113.60,6292.70
5192,1,0,0,0,72,1,1,1,1,1,1,1,1,1,2,0,0,113.40,8164.10
3980,1,0,0,0,63,1,1,1,0,1,0,0,0,1,1,0,1,90.45,5825.50
235,0,0,0,0,22,1,0,2,0,0,0,0,0,0,1,0,0,20.85,450.65


In [24]:
train_y

821     0
1571    0
4355    0
4709    0
4799    0
       ..
905     0
5192    0
3980    0
235     0
5157    0
Name: Churn, Length: 3943, dtype: int32

In [25]:
val_X

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
5244,1,0,0,0,41,1,0,2,0,0,0,0,0,0,2,0,3,20.15,802.35
3194,0,0,1,1,23,1,0,2,0,0,0,0,0,0,1,0,3,20.05,415.10
2107,1,0,0,0,3,1,0,0,1,0,0,1,1,0,0,1,1,64.40,195.65
3847,1,1,0,1,56,1,0,1,1,0,0,0,1,1,1,0,1,94.80,5264.30
566,1,0,1,1,18,1,0,0,1,0,1,1,1,1,0,0,2,83.25,1611.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3162,1,0,0,0,53,1,1,0,0,1,1,1,0,1,2,1,0,73.90,3958.25
3185,1,0,1,0,63,1,0,2,0,0,0,0,0,0,2,1,0,19.95,1234.80
4494,0,0,0,0,18,1,1,0,0,0,0,0,0,0,0,1,2,50.30,913.30
2478,1,0,0,0,5,1,0,0,0,0,1,0,1,1,0,1,2,70.05,346.40


In [26]:
val_y

5244    0
3194    0
2107    0
3847    0
566     0
       ..
3162    0
3185    0
4494    0
2478    1
534     1
Name: Churn, Length: 1691, dtype: int32

In [27]:
# standardizzo i dati in modo che ogni colonna abbia media uguale a 0 e deviazione standard pari a 1
# le prestazioni ottenute sono migliori
sc = StandardScaler()
train_X = pd.DataFrame(sc.fit_transform(train_X))
val_X = pd.DataFrame(sc.fit_transform(val_X))

### Modellazione e Allenamento

In [28]:
from sklearn.linear_model import LogisticRegression
log_reg_model = LogisticRegression(n_jobs=-1)
log_reg_model.fit(train_X, train_y)
# validazione
preds = log_reg_model.predict(val_X)

### Accuracy

In [29]:
# calcolo MAE
mae = mean_absolute_error(val_y, preds)
# calcolo accuracy sul training set in percentuale
train_accuracy = log_reg_model.score(train_X, train_y)*100
# calcolo accuracy sul test set in percentuale
test_accuracy = log_reg_model.score(val_X, val_y)*100
print('Mean absolute error: ')
print(mae)
print('Accuracy training set: ')
print(train_accuracy)
print('Accuracy test set: ')
print(test_accuracy)

Mean absolute error: 
0.1992903607332939
Accuracy training set: 
80.14202383971595
Accuracy test set: 
80.07096392667061
